In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import bz2
import gc
import chardet
import re

In [2]:
train_file = bz2.BZ2File('train.ft.txt.bz2')
test_file = bz2.BZ2File('test.ft.txt.bz2')

In [3]:
#reading the data and appending it to python-list
train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()
del train_file, test_file
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]

In [4]:
#Seprating scores from reviews
length_of_train = len(train_file_lines)
length_of_test = len(test_file_lines)
train_file_lines_score = []
test_file_lines_score = []
for i in range(0,length_of_train):
    temp = train_file_lines[i][9]
    train_file_lines_score.append(temp)

for i in range(0,length_of_test):
    temp = test_file_lines[i][9]
    test_file_lines_score.append(temp)
print(length_of_train , length_of_test , len(train_file_lines_score) , len(test_file_lines_score))

3600000 400000 3600000 400000


In [5]:
#Converting python list to dataframe
df_train_reviews = pd.DataFrame({'reviews': train_file_lines} )
df_test_reviews = pd.DataFrame({'reviews': test_file_lines} )
df_train_score = pd.DataFrame({'score': train_file_lines_score} )
df_test_score = pd.DataFrame({'score': test_file_lines_score} )

#taking less reviews due to limited computational power
df_train_reviews = df_train_reviews[:100000]
df_test_reviews = df_test_reviews[:25000]
df_train_score = df_train_score[:100000]
df_test_score = df_test_score[:25000]


print(df_train_reviews.shape,df_test_reviews.shape,df_train_score.shape,df_test_score.shape)

(100000, 1) (25000, 1) (100000, 1) (25000, 1)


In [6]:
#pre-processing data
import re
import nltk
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

#Pre-processing reviews data of traning set
corpus_train = []
for i in range(0, len(df_train_reviews)):
    review = re.sub("<.*?>", "", df_train_reviews['reviews'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    del review[0]
    #ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer() 
    #review = [ps.stem(words) for words in review if not words in set(stopwords.words('english'))]
    review = [lemmatizer.lemmatize(words) for words in review if not words in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_train.append(review)
    
    #Pre-processing reviews data of testing set
corpus_test = []
for i in range(0, len(df_test_reviews)):
    review = re.sub("<.*?>", "", df_test_reviews['reviews'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    del review[0]
    #ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer() 
    #review = [ps.stem(words) for words in review if not words in set(stopwords.words('english'))]
    review = [lemmatizer.lemmatize(words) for words in review if not words in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_test.append(review)

corpus_train[0]

'stuning even non gamer sound track beautiful paint senery mind well would recomend even people hate vid game music played game chrono cross game ever played best music back away crude keyboarding take fresher step grate guitar soulful orchestra would impress anyone care listen'

In [7]:
#Bag of words(sparc matrix)
from sklearn.feature_extraction.text import TfidfVectorizer
#cv = TfidfVectorizer(ngram_range=(1,2))
cv = TfidfVectorizer(max_features=9000)
X_train = cv.fit_transform(corpus_train).toarray()
X_test = cv.transform(corpus_test).toarray()
X_train.shape

(100000, 9000)

In [8]:
#Naive bayes classification
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB(alpha = 1 , fit_prior=True, class_prior=None)
classifier.fit(X_train, df_train_score)
y_pred = classifier.predict(X_test)

D:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [10]:
from sklearn.metrics import confusion_matrix
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements 

#making the confusion matrix
cm = confusion_matrix(df_test_score , y_pred)
print(cm)
accuracy(cm)

[[10106  2051]
 [ 1844 10999]]


0.8442

In [11]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_test_score , y_pred)
accuracy

0.8442